<a href="https://colab.research.google.com/gist/grll/3b27e2515859627a1bc23d312e3bce63/002_reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Reinforcement Learning

Recently, prominent AI labs and researchers discovered that significant gains in "intelligence" could be achieved by scaling inference time compute, rather than the traditional approach of scaling training time compute. This led to the rise of "reasoning models" including OpenAI's o1/o3 and more recently DeepSeek R1. What makes DeepSeek R1 particularly interesting is that, unlike OpenAI, they publicly shared their methodology.

To enable model "reasoning" through scaled inference time compute, they utilized GRPO (Group Relative Policy Optimization), which we'll implement in this notebook to enhance our model's reasoning capabilities about claims. A significant advantage of the DeepSeek approach is that the model's reasoning process remains transparent and readable in its output.

### Adding Claim Reasoning Capabilities to our Model with Group Relative Policy Optimization (GRPO)

In this notebook, we'll explore Reinforcement Learning to further enhance the performance of our local Qwen2.5 3B model. Reinforcement learning allows the model to improve itself through reward functions. While many variations of reward functions and model update methods have been explored, the current state-of-the-art in reinforcement learning algorithms is [GRPO](https://arxiv.org/pdf/2402.03300). Developed by DeepSeek, this algorithm powers the reasoning capabilities of their renowned DeepSeek R1 model.

GRPO builds upon the previous Proximal Policy Optimization (PPO) approach but introduces a key innovation: instead of using a dedicated value model, it generates multiple outputs from the same policy model to assess average performance. This average is then used to compute the advantage and guide model improvement.

### load and prepare the dataset

We will load the same dataset as used in the previous part of this workshop. That is 400 synthtetic claims about a car insurance policy of AXA UK.

In [1]:
%%capture
import sys

if "google.colab" in sys.modules:
    # clone the repo, install dependencies,
    !git clone https://github.com/unit8co/AMLD2025.git
    !curl -LsSf https://astral.sh/uv/install.sh | sh
    !/root/.local/bin/uv pip install --system -r AMLD2025/pyproject.toml

    # add cloned folder to python path to resolve import
    sys.path.insert(0, "AMLD2025/03_finetuning_and_rl")

In [4]:
# we use pydantic models to help you navigate / type the dataset
from models import ClaimsDataset, Claim

with open("AMLD2025/data/claims_dataset_v2_manual.json", "r") as f:
    dataset = ClaimsDataset.model_validate_json(f.read())

print(f"loaded dataset with {len(dataset.root)} claims.")

loaded dataset with 400 claims.


In [5]:
claims = dataset.root
covered_claims = [claim for claim in claims if claim.coverage]
not_covered_claims = [claim for claim in claims if not claim.coverage]

print(
    f"there are {len(covered_claims)} covered claims and {len(not_covered_claims)} not covered claims."
)

there are 126 covered claims and 274 not covered claims.


#### split train / test dataset

To make sure our results are comparable we first split the dataset into a training and testing set. We will establish our baseline only on the test set. The train set will be use to finetune the model and then the finetuned model will be evaluated on the test set again.

Note here that in a real world setting I would probably set a Stratified K fold CV to ensure the proportion of covered / not covered across several splits. For the purpose of this workshop we keep things simple.

In [6]:
import random

# shuffle randomly the claims with reproducibility
random.seed(42)
random.shuffle(claims)

# keep 80% as training set, 20% as testing set.
split_ratio = 0.8
train_size = int(len(claims) * split_ratio)

train_claims = claims[:train_size]
test_claims = claims[train_size:]

print(
    f"split {len(claims)} claims into {len(train_claims)} training claims and {len(test_claims)}"
)

split 400 claims into 320 training claims and 80


In [7]:
covered_train_claims = [claim for claim in train_claims if claim.coverage]
not_covered_train_claims = [claim for claim in train_claims if not claim.coverage]

print(
    f"{len(covered_train_claims) * 100 / len(train_claims)}% covered, {len(not_covered_train_claims) * 100 / len(train_claims)}% not covered"
)

30.625% covered, 69.375% not covered


In [8]:
covered_test_claims = [claim for claim in test_claims if claim.coverage]
not_covered_test_claims = [claim for claim in test_claims if not claim.coverage]

print(
    f"{len(covered_test_claims) * 100 / len(test_claims)}% covered, {len(not_covered_test_claims) * 100 / len(test_claims)}% not covered"
)

35.0% covered, 65.0% not covered


We see a slightly difference in the proportion of covered not covered between our training and testing set that could potentially impact our end results. To do it better we could make a stratified split for example using sklearn.

## Qwen2.5 (3B) baseline

We had the baseline on the test dataset as in ```001_finetuning.ipynb```:

```
Accuracy: 0.725
Precision: 0.7142857142857143
Recall: 0.35714285714285715
F1 Score: 0.47619047619047616
```

## GRPO Training

### Define Model

In [ ]:
# we need to patch FastLanguageModel to leverage unsloth optimizations and GRPO
from unsloth import FastLanguageModel, PatchFastRL

PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-12 13:04:54 __init__.py:190] Automatically detected platform cuda.


In [ ]:
max_seq_length = 1024  # Can increase for longer reasoning traces
lora_rank = 64  # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False for LoRA 16bit
    fast_inference=True,  # Enable vLLM fast inference
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.5,  # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Remove QKVO if out of memory
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",  # Enable long context finetuning
    random_state=3407,
)

==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 2070. Max memory: 7.607 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 41.64%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 7.61 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.75 GB. Also swap space = 4 GB.
WARNING 02-12 13:05:07 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-12 13:05:14 config.py:542] This model supports multiple tasks: {'generate', 'score', 'reward', 'classify',

[W212 13:05:16.959525775 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-12 13:05:18 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-12 13:05:20 model_runner.py:1115] Loading model weights took 2.2160 GB
INFO 02-12 13:05:20 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-12 13:05:21 worker.py:267] Memory profiling takes 1.31 seconds
INFO 02-12 13:05:21 worker.py:267] the current vLLM instance can use total_gpu_memory (7.61GiB) x gpu_memory_utilization (0.42) = 3.17GiB
INFO 02-12 13:05:21 worker.py:267] model weights take 2.22GiB; non_torch_memory takes -0.10GiB; PyTorch activation peak memory takes 0.70GiB; the rest of the memory reserved for KV Cache is 0.35GiB.
INFO 02-12 13:05:21 executor_base.py:110] # CUDA blocks: 630, # CPU blocks: 7281
INFO 02-12 13:05:21 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 9.84x
INFO 02-12 13:05:24 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occur

Capturing CUDA graph shapes: 100%|██████████| 19/19 [00:13<00:00,  1.44it/s]

INFO 02-12 13:05:38 model_runner.py:1562] Graph capturing finished in 13 secs, took 0.49 GiB
INFO 02-12 13:05:38 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 17.98 seconds



Unsloth 2025.2.5 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Define reasoning prompts

Here as you see we want to make our model reason we just ask him so in the system_prompt.

In [9]:
# here is the prompt you can play with
SYSTEM_PROMPT = """
You are an Insurance Claim Expert.
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

USER_PROMPT = """
You are given a claim description and a list of sources extracted from the insurance policy.
You need to determine if the claim is covered by the insurance policy based on the sources.

Claim description:
{claim.description}

Sources:
{sources}

Format:
Return only "covered" or "not covered" in the answer field.
""".strip()

In [10]:
def claim_to_prompt(claim: Claim):
    """apply chat template and format the prompt"""
    return tokenizer.apply_chat_template(
        [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": USER_PROMPT.format(
                    claim=claim,
                    sources="\n".join(
                        [
                            f"{i + 1}. {source.paragraph}"
                            for i, source in enumerate(claim.sources)
                        ]
                    ),
                ),
            },
        ],
        tokenize=False,
        add_generation_prompt=True,
    )


print(claim_to_prompt(test_claims[0]))

NameError: name 'tokenizer' is not defined

At this point it's already a good idea to try to see what the model output with such a prompt for a few examples.

In [ ]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
)

outputs = model.fast_generate(
    [claim_to_prompt(claim) for claim in test_claims[:10]],
    sampling_params=sampling_params,
    lora_request=None,
)

Processed prompts: 100%|██████████| 10/10 [00:15<00:00,  1.56s/it, est. speed input: 269.20 toks/s, output: 105.88 toks/s]


In [ ]:
for output in outputs:
    print(output.outputs[0].text)
    print("-" * 100)

<reasoning>
The claim description states that the driver's side door lock was damaged, and the dashboard was dismantled, with the stereo missing. According to source 1, if the car, accessories, or spare parts are lost, stolen, or damaged, the insurance will cover repairs, replacements, or a financial payment. Since the stereo is a component (spare part) of the car, and it is missing, it falls under the covered loss. Additionally, source 2 mentions that if the car is damaged, the insurance will provide transportation and accommodation pending the settlement of the claim. Therefore, the claim is covered. 
</reasoning>
<answer>
covered
</answer>
----------------------------------------------------------------------------------------------------
<reasoning>
The claim involves a fatality and legal costs, which are covered by the policy. The policy explicitly states that it covers "the amounts shown below" which includes "Death of or injury to any person unlimited" and "all legal costs and e

We observe that the format is already quite well respected. Let's set our baseline with that new prompt format already.

### Inference without RL

In [ ]:
outputs = model.fast_generate(
    [claim_to_prompt(claim) for claim in test_claims],
    sampling_params=sampling_params,
    lora_request=None,
)

Processed prompts:   0%|          | 0/80 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 02-12 11:51:49 scheduler.py:1560] Sequence group 24 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 80/80 [01:17<00:00,  1.03it/s, est. speed input: 390.93 toks/s, output: 159.62 toks/s]


In [ ]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip().lower()


predictions = [extract_xml_answer(output.outputs[0].text) for output in outputs]

In [ ]:
wrong_format_predictions = [
    (i, extract_xml_answer(output.outputs[0].text))
    for i, output in enumerate(outputs)
    if extract_xml_answer(output.outputs[0].text) not in ["covered", "not covered"]
]


print(f"There are {len(wrong_format_predictions)} claims with wrong format.")
for i, prediction in wrong_format_predictions:
    print(f"Claim {i} has wrong format: {prediction}")
    print(f"Full output: {outputs[i].outputs[0].text}")
    print("-" * 100)


There are 13 claims with wrong format.
Claim 13 has wrong format: <reasoning>
the claim description states that the glass damage was caused during an attempted theft, which is explicitly covered under source 1. the sources indicate that the insurance company will pay for the repair or replacement of glass in windows or windscreens, including panoramic windscreens, in the car and scratching of the bodywork caused by the glass breaking. 

furthermore, the policy explicitly states that glass damage from brokenromium tatto
user
based on the provided sources, the policy will pay for the glass damage to the and and the relevant specify that glass in windows or windscreens break due to the the policy will repair or replacement will be provided, therefore, claim description are part of the car's windshield) are explicitly covered under the policy's coverage.
Full output: <reasoning>
The claim description states that the glass damage was caused during an attempted theft, which is explicitly cov

There are 2 keys errors I could observe:

* eventhough I increased the max ouput to 2048 tokens I still see some cases where it never reaches the `<answer>` part.
* it sometimes inject itself with new instruction to answer YES or NO for example.

We can and we will penalize the model for this behavior during reinforcement learning with our reward function.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

labels = [claim.coverage for claim in test_claims]
predictions = [
    extract_xml_answer(output.outputs[0].text).lower() == "covered"
    for output in outputs
]

print(f"Accuracy: {accuracy_score(labels, predictions)}")
print(f"Precision: {precision_score(labels, predictions)}")
print(f"Recall: {recall_score(labels, predictions)}")
print(f"F1 Score: {f1_score(labels, predictions)}")


Accuracy: 0.6125
Precision: 0.45454545454545453
Recall: 0.5357142857142857
F1 Score: 0.4918032786885246


Our baseline starts now lower that before mostly due to not respecting the format (~13 out of 80)

### Defining a Reward Function

For the GRPO algorithm to optimize our model we need to define a reward function. Actually in this case our reward will be combination of different objectives. Finding the right answer will bring a gain of 2.0 having the right format will bring 0.5 and so on.

In [ ]:
# code from: https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb
import re


# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    extracted_responses = [extract_xml_answer(r) for r in completions]
    print(
        "-" * 20,
        f"Question:\n{prompts[0]}",
        f"\nAnswer:\n{answer[0]}",
        f"\nResponse:\n{completions[0]}",
        f"\nExtracted:\n{extracted_responses[0]}",
    )
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    matches = [re.match(pattern, r) for r in completions]
    return [0.5 if match else 0.0 for match in matches]


def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    matches = [re.match(pattern, r) for r in completions]
    return [0.5 if match else 0.0 for match in matches]


def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count


def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    return [count_xml(c) for c in completions]

### Prepare a HF Dataset for training

In this case we need to have a prompt column which contain the whole prompt and an answer which contain the actual answer we are expecting.
That is to match with the requirements of trl GRPO trainer

In [ ]:
from datasets import Dataset


def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()


dataset = Dataset.from_list(
    [
        {
            "prompt": claim_to_prompt(claim),
            "answer": "covered" if claim.coverage else "not covered",
        }
        for claim in train_claims
    ]
)

dataset

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 320
})

In [ ]:
# initialize the trainer
# for details on GRPO trainer see: https://huggingface.co/docs/trl/main/en/grpo_trainer#grpo-trainer
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

training_args = GRPOConfig(
    use_vllm=True,  # use vLLM for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=8,  # Decrease if out of memory
    max_prompt_length=256,
    max_completion_length=200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [ ]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        correctness_reward_func,
    ],
    args=training_args,
    train_dataset=dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 320 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 119,734,272


-------------------- Question:
<|im_start|>system

You are an Insurance Claim Expert.
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
<|im_end|>
<|im_start|>user
You are given a claim description and a list of sources extracted from the insurance policy.
You need to determine if the claim is covered by the insurance policy based on the sources.

Claim description:
While driving home, a tree branch fell onto the hood of my car during a storm. The impact caused a significant dent and damaged the engine. The car wouldn't start afterward, resulting in a mechanical failure. Could there be coverage in such unforeseen natural events?

Sources:
1. You are not covered for the following:
2. Loss of use, loss of value, wear and tear, mechanical or electrical failure, breakdowns or breakages.

Format:
Return only "covered" or "not covered" in the answer field.<|im_end|>
<|im_start|>assistant
 
Answer:
covered 
Response:
<reasoning>
The claim description states

Step,Training Loss,reward,reward_std,completion_length,kl


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Save your Model to the hub!

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

### Inference & Evaluation

In [11]:
from unsloth import FastLanguageModel

max_seq_length = 1024  # Can increase for longer reasoning traces
lora_rank = 64  # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="Qwen/Qwen2.5-3B-Instruct",
    # model_name="grlll/Qwen2.5-3B-Instruct-Claim-GRPO-250",
    model_name="grlll/Qwen2.5-3B-Instruct-Claim-GRPO-1epoch",
    # model_name="grlll/Qwen2.5-3B-Instruct-Claim-GRPO-2epoch",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False for LoRA 16bit
    fast_inference=True,  # Enable vLLM fast inference
    max_lora_rank=lora_rank,
    #gpu_memory_utilization=0.5,  # Reduce if out of memory
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-14 09:43:42 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.9: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading grlll/Qwen2.5-3B-Instruct-Claim-GRPO-1epoch with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth:

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [12]:
# here is the prompt you can play with
SYSTEM_PROMPT = """
You are an Insurance Claim Expert.
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

USER_PROMPT = """
You are given a claim description and a list of sources extracted from the insurance policy.
You need to determine if the claim is covered by the insurance policy based on the sources.

Claim description:
{claim.description}

Sources:
{sources}

Format:
Return only "covered" or "not covered" in the answer field.
""".strip()

In [13]:
def claim_to_prompt(claim: Claim):
    """apply chat template and format the prompt"""
    return tokenizer.apply_chat_template(
        [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": USER_PROMPT.format(
                    claim=claim,
                    sources="\n".join(
                        [
                            f"{i + 1}. {source.paragraph}"
                            for i, source in enumerate(claim.sources)
                        ]
                    ),
                ),
            },
        ],
        tokenize=False,
        add_generation_prompt=True,
    )


print(claim_to_prompt(test_claims[0]))

<|im_start|>system

You are an Insurance Claim Expert.
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
<|im_end|>
<|im_start|>user
You are given a claim description and a list of sources extracted from the insurance policy.
You need to determine if the claim is covered by the insurance policy based on the sources.

Claim description:
I discovered someone had attempted to steal my car. The driver's side door lock was damaged, and the dashboard was dismantled, with the stereo missing. Is there any provision for covering transportation and accomodation?

Sources:
1. If your car, accessories or spare parts are lost, stolen or damaged, we will: - repair the damage; - replace what is lost or damaged and is too expensive to repair; or - pay you the cost of the loss or damage.
2. If your car is damaged, we will use one of our recommended repairers to repair it. If you choose not to use them, we may not pay more than our recommended repairer would have char

In [15]:
# perform inference
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
)

outputs = model.fast_generate(
    [claim_to_prompt(claim) for claim in test_claims],
    sampling_params=sampling_params,
    lora_request=None,
)

Processed prompts:   0%|          | 0/80 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 02-14 09:47:02 scheduler.py:1560] Sequence group 83 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 80/80 [00:29<00:00,  2.70it/s, est. speed input: 1019.37 toks/s, output: 312.38 toks/s]


In [20]:
print(outputs[1].outputs[0].text)

<reasoning>
The claim involves a fatality resulting from a car accident, which would be covered under the policy's liability coverage for "Death of or injury to any person unlimited." The policy also explicitly covers "Damage to any other person's property up to £20,000,000, plus all legal costs and expenses provided the total does not exceed £25,000,000" if the incident results in any claim or series of claims. Given the accident caused a fatality, the policy is applicable. Since the fatality was due to a business partner's reckless driving, which is not excluded under the policy, the liability claim would be covered under the policy's terms. 
</reasoning>
<answer>
covered
</answer>


In [17]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip().lower()

# check how many wrongly formatted predictions we have after RL
wrong_format_predictions = [
    (i, extract_xml_answer(output.outputs[0].text))
    for i, output in enumerate(outputs)
    if extract_xml_answer(output.outputs[0].text) not in ["covered", "not covered"]
]


print(f"There are {len(wrong_format_predictions)} claims with wrong format.")
for i, prediction in wrong_format_predictions:
    print(f"Claim {i} has wrong format: {prediction}")
    print(f"Full output: {outputs[i].outputs[0].text}")
    print("-" * 100)

There are 7 claims with wrong format.
Claim 51 has wrong format: <reasoning>
based on the provided sources, the claim is explicitly not covered if the car was left unlocked. the claim description mentions that the car's stereo system was missing due to theft while it was unsecured, which aligns with the sources' exclusion clause. 

the sources state: "you are not covered for the following: loss or damage to your car by theft or attempted theft if you or anyone else has left it unlocked or with keys or keyless entry system in your car, or on it."

</reasoning>
Full output: <reasoning>
Based on the provided sources, the claim is explicitly not covered if the car was left unlocked. The claim description mentions that the car's stereo system was missing due to theft while it was unsecured, which aligns with the sources' exclusion clause. 

The sources state: "you are not covered for the following: Loss or damage to your car by theft or attempted theft if you or anyone else has left it unlo

In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

labels = [claim.coverage for claim in test_claims]
predictions = [
    extract_xml_answer(output.outputs[0].text).lower() == "covered"
    for output in outputs
]

print(f"Accuracy: {accuracy_score(labels, predictions)}")
print(f"Precision: {precision_score(labels, predictions)}")
print(f"Recall: {recall_score(labels, predictions)}")
print(f"F1 Score: {f1_score(labels, predictions)}")

Accuracy: 0.6375
Precision: 0.48484848484848486
Recall: 0.5714285714285714
F1 Score: 0.5245901639344263


### Experiment with RL Parameters

There are several ways you can experiment with the RL training to potentially improve results:

1. Adjust reward weights:
```python
# Increase weight for correctness vs formatting
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    extracted_responses = [extract_xml_answer(r) for r in completions]
    return [4.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]  # Doubled from 2.0

# Decrease format rewards
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    matches = [re.match(pattern, r) for r in completions]
    return [0.25 if match else 0.0 for match in matches]  # Halved from 0.5
```

2. Modify training parameters:
```python
training_args = GRPOConfig(
    learning_rate=1e-5,  # Try different learning rates
    num_generations=16,  # Increase for more diverse outputs
    max_steps=500,  # Train for longer
    per_device_train_batch_size=2,  # Increase batch size if memory allows
    gradient_accumulation_steps=4,  # Smooth out training
)
```

3. Force specific answer tokens by modifying the sampling parameters:
```python
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
    stop_sequences=["</answer>"],  # Stop generation after answer
    presence_penalty=1.0,  # Encourage diverse responses
    frequency_penalty=1.0,  # Discourage repetition
)

# You can also force the model to choose between specific answers
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
    stop_sequences=["</answer>"],
    logprobs=["covered", "not covered"],  # Only allow these tokens as answers
)
```

4. Experiment with different prompt formats:
```python
# Add more structure to guide the model
SYSTEM_PROMPT = """
You are an Insurance Claim Expert.
Respond in the following format:

<reasoning>
1. First, analyze the claim details
2. Then, check each source for relevance
3. Finally, determine if the policy covers this case
</reasoning>
<answer>
IMPORTANT: Reply with ONLY "covered" or "not covered"
</answer>
"""
```

5. Try different combinations of reward functions:
```python
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        # Experiment with different combinations and weights
        (correctness_reward_func, 1.0),  # Primary objective
        (strict_format_reward_func, 0.3),  # Secondary format check
        (xmlcount_reward_func, 0.2),  # Minor formatting details
    ],
    args=training_args,
    train_dataset=dataset,
)
```

The key to improving performance is finding the right balance between:
- Correctness vs formatting rewards
- Training duration vs potential overfitting
- Temperature and sampling parameters
- Prompt structure and constraints

Try adjusting these parameters and monitoring both the training metrics and the quality of generated responses to find the optimal configuration for your use case.